# Полная интеграция: От правил к математическим структурам

В этом туториале мы используем **SpaceLanguageOrchestrator** — центральный компонент, объединяющий все 8 уровней анализа (L0-L7).

## 🎯 Что вы узнаете

1. Как настроить полный пайплайн анализа
2. Как запустить анализ на всех уровнях
3. Как интерпретировать результаты
4. Как сохранить и загрузить результаты

## 📦 Архитектура

```
L0: Rewriting System → L1: Turing Machine → L2: Graphs & SCC
                                                     ↓
L3: AC-automata ← L4: Macros ← L5: Text Generation ←┘
                      ↓
L6: Verification (Z3) + L7: Math Lifts (Geometry, Measures, Categories)
```

## 🚀 Настройка окружения

In [1]:
import sys
from pathlib import Path
import importlib

# Настройка путей
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Принудительная перезагрузка модулей
import integration.orchestrator
importlib.reload(integration.orchestrator)

from rewriting import String, Rule
from integration.orchestrator import SpaceLanguageOrchestrator, PipelineConfig

print("✓ Интеграционные модули загружены (с перезагрузкой)")
print(f"✓ Project root: {project_root}")

✓ Интеграционные модули загружены (с перезагрузкой)
✓ Project root: /home/catman/Yandex.Disk/cuckoo/z/reals/libs/Experiments/Space/Lang


## 1️⃣ Определение правил переписывания

Создадим простую систему с тремя правилами:

In [2]:
# Определяем правила - циклическая система с богатым поведением
rules = [
    Rule(String.from_str("00"), String.from_str("|")),    # Сжатие нулей
    Rule(String.from_str("||"), String.from_str("0")),    # Слияние палочек  
    Rule(String.from_str("0|"), String.from_str("|0")),   # Перестановка
    Rule(String.from_str("|0"), String.from_str("0|")),   # Обратная перестановка (создаёт циклы!)
]

print("Правила системы:")
for i, rule in enumerate(rules, 1):
    print(f"  R{i}: {rule}")
    
# Начальная строка - берём длинную с разнообразием для богатых результатов
initial = String.from_str("00|00||0")
print(f"\n✓ Начальная строка: {initial} (длина: {len(initial.symbols)})")
print("  (эта система содержит циклические зависимости)")

Правила системы:
  R1: 00 → |
  R2: || → 0
  R3: 0| → |0
  R4: |0 → 0|

✓ Начальная строка: 00|00||0 (длина: 8)
  (эта система содержит циклические зависимости)


## 2️⃣ Конфигурация пайплайна

`PipelineConfig` управляет всеми параметрами анализа:

In [3]:
config = PipelineConfig(
    # L2: Графовый анализ
    graph_depth=10,  # Увеличена глубина для богатых результатов
    max_graph_size=200,
    
    # L3: AC-автоматы
    min_overlap_length=1,
    
    # L4: Макросы
    min_macro_frequency=2,
    min_macro_length=2,
    max_macro_length=6,
    
    # L6: Верификация (отключена - требует z3-solver)
    enable_z3=False,
    z3_timeout=3000,
    enable_isabelle=False,
    
    # L7: Математические подъёмы
    enable_geometry=True,
    enable_measure=True,
    enable_category=True,
    embedding_dim=10,
)

print("✓ Конфигурация создана")
print(f"  Глубина графа: {config.graph_depth}")
print(f"  Размерность погружения: {config.embedding_dim}")
print(f"  Z3 верификация: {'включена' if config.enable_z3 else 'отключена (требует z3-solver)'}")

✓ Конфигурация создана
  Глубина графа: 10
  Размерность погружения: 10
  Z3 верификация: отключена (требует z3-solver)


## 3️⃣ Создание оркестратора

`SpaceLanguageOrchestrator` координирует работу всех компонентов:

In [4]:
# Создаём оркестратор
orchestrator = SpaceLanguageOrchestrator(rules, config)

print("✓ Оркестратор инициализирован")
print(f"  Правил: {len(rules)}")
print(f"  Компонентов: L0-L7 (8 уровней)")

✓ Оркестратор инициализирован
  Правил: 4
  Компонентов: L0-L7 (8 уровней)


## 4️⃣ Запуск полного анализа

Анализ проходит через все 8 уровней (это может занять несколько секунд):

In [5]:
# Запускаем полный анализ
result = orchestrator.analyze(initial, verbose=True)

print("\n" + "="*60)
print("✓ АНАЛИЗ ЗАВЕРШЁН")
print("="*60)

L0: Анализ системы переписывания...
L2: Построение графа и SCC...
L3: Анализ перекрытий...
L4: Извлечение макросов...
L5: Генерация текста...
L7: Математические подъёмы...

✓ АНАЛИЗ ЗАВЕРШЁН


## 5️⃣ Краткое резюме

Получаем краткую сводку результатов:

In [6]:
# Краткая сводка
print(result.summary())

# Если были ошибки - покажем их
if result.errors:
    print("\n⚠️ Детали ошибок:")
    for e in result.errors:
        print(f"  {e}")

РЕЗУЛЬТАТЫ АНАЛИЗА SPACE LANGUAGE

Правила: 4
Начальная строка: 00|00||0

L0: Система переписывания
  • Достижимых строк: 140
  • ω-предел: 1 строк
  • Нормальных форм: 1

L2: Графовый анализ
  • Вершин: 140
  • Рёбер: 815
  • SCC компонент: 12
  • Наибольшая SCC: 56 вершин

L3: Перекрытия
  • Пар перекрытий: 5
  • Критических пар: 1

L4: Макросы
  • Найдено макросов: 20
  • Покрытие: 99.3%

L5: Генерация текста
  • AST узлов: 5
  • Длина текста: 117 символов

L7: Математические подъёмы
  • Геометрия: размерность=2
  • Меры: энтропия=7.129 бит
  • Категория: объектов=30

Время выполнения: 0.14 сек
Ошибок: 0
Предупреждений: 0


## 6️⃣ Детальный анализ результатов

### L0: Система переписывания

In [7]:
print("L0: СИСТЕМА ПЕРЕПИСЫВАНИЯ")
print(f"  Достижимых строк: {len(result.reachable_strings)}")
print(f"  Нормальных форм: {len(result.normal_forms)}")
print(f"  ω-предел: {len(result.omega_limit)} строк")

print("\n  Примеры достижимых строк:")
for s in list(result.reachable_strings)[:5]:
    print(f"    {s}")

L0: СИСТЕМА ПЕРЕПИСЫВАНИЯ
  Достижимых строк: 140
  Нормальных форм: 1
  ω-предел: 1 строк

  Примеры достижимых строк:
    00|||000
    |000|0
    0|||0
    0||000
    0|000|0|


### L2: Графовый анализ

In [8]:
print("L2: ГРАФОВЫЙ АНАЛИЗ")
print(f"  Вершин графа: {result.graph_vertices}")
print(f"  Рёбер: {result.graph_edges}")
print(f"  SCC компонент: {result.scc_count}")
print(f"  Наибольшая SCC: {result.largest_scc_size} вершин")

L2: ГРАФОВЫЙ АНАЛИЗ
  Вершин графа: 140
  Рёбер: 815
  SCC компонент: 12
  Наибольшая SCC: 56 вершин


### L4: Макросы

In [9]:
print("L4: МАКРОСЫ")
print(f"  Найдено макросов: {len(result.macros)}")

if result.macros and len(result.macros) > 0:
    print("\n  Топ-3 макроса по частоте:")
    for i, macro in enumerate(list(result.macros)[:3], 1):
        print(f"    {i}. {macro}")
else:
    print("  (макросы не обнаружены)")

L4: МАКРОСЫ
  Найдено макросов: 20

  Топ-3 макроса по частоте:
    1. A := 0| ?
    2. B := || ?
    3. C := |0 ?


### L7: Математические подъёмы

In [10]:
print("L7: МАТЕМАТИЧЕСКИЕ ПОДЪЁМЫ")

# Геометрия
if result.geometry_data:
    print("\n  📐 Геометрия:")
    print(f"    Размерность: {result.geometry_data.get('dimension', 'N/A')}")
    print(f"    Кривизна: {result.geometry_data.get('curvature', 0):.4f}")
    print(f"    Точек: {result.geometry_data.get('points', 0)}")

# Меры
if result.measure_data:
    print("\n  📊 Меры:")
    print(f"    Энтропия: {result.measure_data.get('entropy', 0):.3f} бит")
    print(f"    Размер носителя: {result.measure_data.get('support_size', 0)}")

# Категории
if result.category_data:
    print("\n  🎯 Категории:")
    print(f"    Объектов: {result.category_data.get('objects', 0)}")
    print(f"    Морфизмов: {result.category_data.get('morphisms', 0)}")

L7: МАТЕМАТИЧЕСКИЕ ПОДЪЁМЫ

  📐 Геометрия:
    Размерность: 2
    Кривизна: 0.0704
    Точек: 20

  📊 Меры:
    Энтропия: 7.129 бит
    Размер носителя: 140

  🎯 Категории:
    Объектов: 30
    Морфизмов: 30


## 7️⃣ Экспорт результатов

Сохраняем результаты в JSON:

In [11]:
import json

# Получаем словарь
result_dict = result.to_dict()

# Сохраняем в файл
output_path = project_root / "results_integration.json"
result.save_json(str(output_path))

print(f"✓ Результаты сохранены: {output_path}")
print(f"  Размер: {output_path.stat().st_size / 1024:.1f} KB")

# Показываем ключи верхнего уровня
print(f"\n  Разделы результатов:")
for key in list(result_dict.keys())[:10]:
    print(f"    - {key}")

✓ Результаты сохранены: /home/catman/Yandex.Disk/cuckoo/z/reals/libs/Experiments/Space/Lang/results_integration.json
  Размер: 1.7 KB

  Разделы результатов:
    - rules
    - initial_string
    - config
    - rewriting
    - graph
    - overlaps
    - macros
    - text
    - verification
    - math_lifts


## 📊 Резюме

В этом туториале вы научились:

✅ **Настраивать** полный пайплайн через `PipelineConfig`  
✅ **Запускать** анализ на всех 8 уровнях с помощью `SpaceLanguageOrchestrator`  
✅ **Интерпретировать** результаты каждого уровня  
✅ **Экспортировать** результаты в JSON  

## 🎯 Ключевые выводы

- **Оркестратор** автоматически координирует работу всех компонентов
- **Конфигурация** позволяет гибко управлять параметрами анализа
- **Результаты** содержат данные от дискретного (L0) до непрерывного (L7) уровней
- **Экспорт** в JSON обеспечивает воспроизводимость

## 🚀 Что дальше?

**04_advanced_examples.ipynb** — Продвинутые примеры и нетривиальные системы